# EDA - Weather Forecast

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose

- Aseguramos al hacer la importación que las fechas sean de tipo Datetime y los decimales en caso de tener comas se sustituya por puntos.

In [2]:
df = pd.read_csv('../Data/raw/open_meteo/historico_provincias_espana.csv')

In [3]:
df.head(5)

,time,temperature_2m_max,temperature_2m_min,temperature_2m_mean,precipitation_sum,windspeed_10m_max,date,provincia
0,2013-01-01,12.3,9.3,10.8,1.2,19.8,2013-01-01,A Coruña
1,2013-01-02,12.4,6.0,9.1,0.0,20.4,2013-01-02,A Coruña
2,2013-01-03,16.1,6.4,10.1,0.0,17.1,2013-01-03,A Coruña
3,2013-01-04,15.2,5.0,9.0,0.0,11.3,2013-01-04,A Coruña
4,2013-01-05,14.2,4.1,8.2,0.0,9.7,2013-01-05,A Coruña


- Vemos la información general de nuestro dataset.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235235 entries, 0 to 235234
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   time                 235235 non-null  object 
 1   temperature_2m_max   235182 non-null  float64
 2   temperature_2m_min   235182 non-null  float64
 3   temperature_2m_mean  235129 non-null  float64
 4   precipitation_sum    235129 non-null  float64
 5   windspeed_10m_max    235182 non-null  float64
 6   date                 235235 non-null  object 
 7   provincia            235235 non-null  object 
dtypes: float64(5), object(3)
memory usage: 14.4+ MB


- Borramos la columna *date*

In [5]:
df.drop(columns=['date'], inplace=True)

- Comprobamos los valores nulos.

In [6]:
df.isna().sum()

time                     0
temperature_2m_max      53
temperature_2m_min      53
temperature_2m_mean    106
precipitation_sum      106
windspeed_10m_max       53
provincia                0
dtype: int64

In [7]:
filas_con_nulos = df[df.isnull().any(axis=1)]
print(len(filas_con_nulos['provincia'].value_counts()))

52


In [8]:
df.dropna(inplace=True)

- Previamente vemos que hay diferentes datos en un mismo día sin tener en cuenta las horas, por lo que realizamos una agrupación de los datos por día segun la media.

In [9]:
df.shape

(235129, 7)

## Histograma

- Extraemos la lista de provincias

In [10]:
provincias = df['provincia'].unique()
print(len(provincias))

52


In [11]:
df_diario_madrid = df[df['provincia'] == 'Madrid']

In [12]:
df_diario_madrid.sort_values(by='time', ascending=True, inplace=True)

C:\Users\aleja\AppData\Local\Temp\ipykernel_19408\3593503295.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_diario_madrid.sort_values(by='time', ascending=True, inplace=True)


### Temperatura media

In [13]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = df_diario_madrid['time'],
        y = df_diario_madrid['temperature_2m_mean'],
        mode = 'lines',
        name = "MADRID"
    )
)

fig.update_layout(
    title = f"Evolución temporal de temperatura media para Madrid",
    xaxis_title = "Fecha",
    yaxis_title = "Temperatura Media (temperature_2m_mean)"
)

fig.show()

### Temperatura máxima

In [14]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = df_diario_madrid['time'],
        y = df_diario_madrid['temperature_2m_max'],
        mode = 'lines',
        name = "MADRID"
    )
)

fig.update_layout(
    title = f"Evolución temporal de temperatura máxima para Madrid",
    xaxis_title = "Fecha",
    yaxis_title = "Temperatura Máxima (temperature_2m_max)"
)

fig.show()

### Temperatura mínima

In [15]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = df_diario_madrid['time'],
        y = df_diario_madrid['temperature_2m_min'],
        mode = 'lines',
        name = "MADRID"
    )
)

fig.update_layout(
    title = f"Evolución temporal de temperatura mínima para Madrid",
    xaxis_title = "Fecha",
    yaxis_title = "Temperatura Mínima (temperature_2m_min)"
)

fig.show()

### Velocidad máxima *(km/h)*

In [16]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_diario_madrid['time'],
        y=df_diario_madrid['windspeed_10m_max'],
        name="MADRID",
        mode='lines'
    )
)

fig.update_layout(
    title = f"Evolución temporal de velocidad máxima del viento(km/h) para Madrid",
    xaxis_title = "Fecha",
    yaxis_title = "Velocidad media (windspeed_10m_max)"
)

fig.show()

### Precipitaciones *(mL)*

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_diario_madrid['time'],
        y=df_diario_madrid['precipitation_sum'],
        name="MADRID",
        mode='lines'
    )
)

fig.update_layout(
    title = f"Evolución temporal de precipitaciones en milímetros para Madrid",
    xaxis_title = "Fecha",
    yaxis_title = "Precipitaciones (precipitation_sum)"
)

fig.show()

## Tendencia, estacionalidad y ruido

### Temperatura media

In [18]:
decomposition_temp_mean = seasonal_decompose(df_diario_madrid['temperature_2m_mean'], model='additive', period=365)

#### 1. Tendencia

In [19]:
fig_trend = go.Figure()
fig_trend.add_trace(go.Scatter(x=df_diario_madrid['time'], y=decomposition_temp_mean.trend, mode='lines', name='Tendencia'))
fig_trend.update_layout(title="Tendencia de la Temperatura Media", xaxis_title="Fecha", yaxis_title="Temperatura media (°C)")
fig_trend.show()

#### 2. Estacionalidad

In [20]:
fig_trend = go.Figure()
fig_trend.add_trace(go.Scatter(x=df_diario_madrid['time'], y=decomposition_temp_mean.seasonal, mode='lines', name='Estacionalidad'))
fig_trend.update_layout(title="Estacionalidad de la Temperatura Media", xaxis_title="Fecha", yaxis_title="Temperatura media (°C)")
fig_trend.show()

#### 3. Ruido

In [21]:
fig_trend = go.Figure()
fig_trend.add_trace(go.Scatter(x=df_diario_madrid['time'], y=decomposition_temp_mean.resid, mode='lines', name='Ruido'))
fig_trend.update_layout(title="Ruido de la Temperatura Media", xaxis_title="Fecha", yaxis_title="Temperatura media (°C)")
fig_trend.show()

### Temperatura máxima

In [22]:
decomposition_temp_mean = seasonal_decompose(df_diario_madrid['temperature_2m_max'], model='additive', period=365)

#### 1. Tendencia

In [23]:
fig_trend = go.Figure()
fig_trend.add_trace(go.Scatter(x=df_diario_madrid['time'], y=decomposition_temp_mean.trend, mode='lines', name='Tendencia'))
fig_trend.update_layout(title="Tendencia de la Temperatura Máxima", xaxis_title="Fecha", yaxis_title="Temperatura máxima (°C)")
fig_trend.show()

#### 2. Estacionalidad

In [24]:
fig_trend = go.Figure()
fig_trend.add_trace(go.Scatter(x=df_diario_madrid['time'], y=decomposition_temp_mean.seasonal, mode='lines', name='Estacionalidad'))
fig_trend.update_layout(title="Estacionalidad de la Temperatura Máxima", xaxis_title="Fecha", yaxis_title="Temperatura máxima (°C)")
fig_trend.show()

#### 3. Ruido

In [25]:
fig_trend = go.Figure()
fig_trend.add_trace(go.Scatter(x=df_diario_madrid['time'], y=decomposition_temp_mean.resid, mode='lines', name='Ruido'))
fig_trend.update_layout(title="Ruido de la Temperatura Máxima", xaxis_title="Fecha", yaxis_title="Temperatura máxima (°C)")
fig_trend.show()

### Temperatura mínima

In [26]:
decomposition_temp_mean = seasonal_decompose(df_diario_madrid['temperature_2m_min'], model='additive', period=365)

#### 1. Tendencia

In [27]:
fig_trend = go.Figure()
fig_trend.add_trace(go.Scatter(x=df_diario_madrid['time'], y=decomposition_temp_mean.trend, mode='lines', name='Tendencia'))
fig_trend.update_layout(title="Tendencia de la Temperatura Mínima", xaxis_title="Fecha", yaxis_title="Temperatura mínima (°C)")
fig_trend.show()

#### 2. Estacionalidad

In [28]:
fig_trend = go.Figure()
fig_trend.add_trace(go.Scatter(x=df_diario_madrid['time'], y=decomposition_temp_mean.seasonal, mode='lines', name='Estacionalidad'))
fig_trend.update_layout(title="Estacionalidad de la Temperatura Mínima", xaxis_title="Fecha", yaxis_title="Temperatura mínima (°C)")
fig_trend.show()

#### 3. Ruido

In [29]:
fig_trend = go.Figure()
fig_trend.add_trace(go.Scatter(x=df_diario_madrid['time'], y=decomposition_temp_mean.resid, mode='lines', name='Ruido'))
fig_trend.update_layout(title="Ruido de la Temperatura Mínima", xaxis_title="Fecha", yaxis_title="Temperatura mínima (°C)")
fig_trend.show()

### Velocidad máxima del viento *(km/h)*

In [30]:
decomposition_temp_mean = seasonal_decompose(df_diario_madrid['windspeed_10m_max'], model='additive', period=365)

#### 1. Tendencia

In [31]:
fig_trend = go.Figure()
fig_trend.add_trace(go.Scatter(x=df_diario_madrid['time'], y=decomposition_temp_mean.trend, mode='lines', name='Tendencia'))
fig_trend.update_layout(title="Tendencia de la Velocidad Máxima del Viento", xaxis_title="Fecha", yaxis_title="Velocidad Máxima del Viento (km/h)")
fig_trend.show()

#### 2. Estacionalidad

In [32]:
fig_trend = go.Figure()
fig_trend.add_trace(go.Scatter(x=df_diario_madrid['time'], y=decomposition_temp_mean.seasonal, mode='lines', name='Estacionalidad'))
fig_trend.update_layout(title="Estacionalidad de la Velocidad Máxima del Viento", xaxis_title="Fecha", yaxis_title="Velocidad Máxima del Viento (km/h)")
fig_trend.show()

#### 3. Ruido

In [33]:
fig_trend = go.Figure()
fig_trend.add_trace(go.Scatter(x=df_diario_madrid['time'], y=decomposition_temp_mean.resid, mode='lines', name='Ruido'))
fig_trend.update_layout(title="Ruido de la Velocidad Máxima del Viento", xaxis_title="Fecha", yaxis_title="Velocidad Máxima del Viento (km/h)")
fig_trend.show()

### Precipitaciones máximas *(mL)*

## Detección de Outliers